### MAST30034: Applied Data Science Project 1
---
# Data Analysis: Modelling Average Trip Rates Over Time
#### Xavier Travers (1178369)

Self-explanatory

In [ ]:
# imports used throughout this notebook
import sys

# add homemade helpers
sys.path.insert(1, '../../scripts')
import helpers.plot_helpers as ph

# path where the data files are stored
DATA_PATH = '../../data'

In [ ]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

### Grouped by Pick-Up Location

In [ ]:
# read in the aggregated tlc by pickup data
tlc_pu_df = spark.read.parquet(f'{DATA_PATH}/curated/tlc/aggregated/yellow/by_pu')
tlc_pu_df.limit(5)

In [ ]:
# generate a time series graph of daily avg trips vs week index.
ph.time_series(tlc_pu_df.toPandas(), 'num_p100k_trips',
    ylabel = 'Total Trips Per 100\'000 People', logy = True)